In [ ]:
import requests
import pandas as pd

In [ ]:
import requests
import pandas as pd

BASE_URL = "https://api.socialverseapp.com"
HEADERS = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}

def fetch_data(endpoint, headers, params):
    data = []
    while True:
        # Make the API request
        response = requests.get(endpoint, headers=headers, params=params)
        
        # Check for a successful response
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break
        
        # Parse the JSON response
        json_data = response.json()
        
        # Print the response to debug the structure
        print(f"Response JSON: {json_data}")
        
        # Check if 'results' key exists and process accordingly
        if "results" in json_data:
            data.extend(json_data["results"])
        else:
            print("The 'results' key is not found in the response. Available keys:", json_data.keys())
            break
        
        # Handle pagination
        if not json_data.get("next"):  # No next page
            break
        
        # Increment the page number for the next request
        params["page"] += 1
    
    return data



In [ ]:
# Fetch viewed posts
params = {
    "page": 1,
    "page_size": 1000,
    "resonance_algorithm": "resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
}

# Call the function to fetch viewed posts
viewed_posts = fetch_data(f"{BASE_URL}/posts/view", HEADERS, params)

# Convert the data to a DataFrame and save it to a CSV file
viewed_posts_df = pd.DataFrame(viewed_posts)
viewed_posts_df.to_csv("viewed_posts.csv", index=False)
print("Viewed posts saved to viewed_posts.csv")


In [ ]:
viewed_posts_df.head()

In [ ]:
# Fetch liked posts
liked_posts = fetch_data(f"{BASE_URL}/posts/like", HEADERS, params)
liked_posts_df = pd.DataFrame(liked_posts)
liked_posts_df.to_csv("liked_posts.csv", index=False)
print("Liked posts saved to liked_posts.csv")


In [65]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import requests
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize Flask app
app = Flask(__name__)

# API Token for Authorization
FLIC_TOKEN = "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
HEADERS = {"Flic-Token": FLIC_TOKEN}

# Base URL for APIs
BASE_URL = "https://api.socialverseapp.com/"

# Utility function to fetch data from API
def fetch_data(endpoint):
    try:
        response = requests.get(BASE_URL + endpoint, headers=HEADERS)
        response.raise_for_status()
        return response.json()["data"]
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Fetch user interactions and video metadata
def get_user_data(username):
    viewed_posts = fetch_data(f"posts/view?username={username}&page_size=1000")
    liked_posts = fetch_data(f"posts/like?username={username}&page_size=1000")
    inspired_posts = fetch_data(f"posts/inspire?username={username}&page_size=1000")
    rated_posts = fetch_data(f"posts/rating?username={username}&page_size=1000")
    return viewed_posts, liked_posts, inspired_posts, rated_posts

def get_all_posts():
    return fetch_data("posts/summary/get?page_size=1000")

# Recommendation algorithm
def recommend_videos(username, category_id=None, mood=None):
    # Fetch data
    all_posts = pd.DataFrame(get_all_posts())
    viewed, liked, inspired, rated = map(pd.DataFrame, get_user_data(username))
    
    # Preprocess data
    all_posts.fillna("", inplace=True)
    user_history = pd.concat([viewed, liked, inspired, rated], ignore_index=True)
    user_history.drop_duplicates(subset="post_id", inplace=True)
    
    # Filter by category if provided
    if category_id:
        all_posts = all_posts[all_posts["category_id"] == category_id]
    
    # Content-based filtering using TF-IDF
    tfidf = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf.fit_transform(all_posts["description"])
    user_profile = tfidf.transform(user_history["description"].sum())
    cosine_sim = cosine_similarity(user_profile, tfidf_matrix)
    content_recommendations = all_posts.iloc[cosine_sim.argsort()[0, -10:][::-1]]
    
    # Collaborative filtering using user-item matrix
    if not user_history.empty:
        user_item_matrix = pd.pivot_table(user_history, index="username", columns="post_id", values="interaction_score", fill_value=0)
        user_similarity = cosine_similarity(user_item_matrix)
        collaborative_recommendations = user_item_matrix.iloc[user_similarity.argsort()[0, -10:][::-1]]
    else:
        collaborative_recommendations = pd.DataFrame()
    
    # Combine recommendations
    final_recommendations = pd.concat([content_recommendations, collaborative_recommendations]).drop_duplicates().head(10)
    if mood:
        final_recommendations = final_recommendations[final_recommendations["tags"].str.contains(mood, case=False, na=False)]
    
    return final_recommendations.to_dict(orient="records")

# API endpoint for recommendations
@app.route('/feed', methods=['GET'])
def feed():
    username = request.args.get("username")
    category_id = request.args.get("category_id")
    mood = request.args.get("mood")
    
    if not username:
        return jsonify({"error": "Username is required"}), 400
    
    recommendations = recommend_videos(username, category_id, mood)
    return jsonify(recommendations)

# Main entry point
if __name__ == "__main__":
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1